Dataset contains Reviews of movies and a binary classification of the movies as output. The words of the reviews are indices of a dictionary.

In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import matplotlib.pyplot as plt

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords

/Users/franzi/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package punkt to /Users/franzi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/franzi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from keras.datasets import imdb

(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=1000,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

In [3]:
#import keras
#NUM_WORDS=1000 # only use top 1000 words
#INDEX_FROM=3   # word index offset

#train,test = keras.datasets.imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)
#train_x,train_y = train
#test_x,test_y = test

#word_to_id = keras.datasets.imdb.get_word_index()
#word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
#word_to_id["<PAD>"] = 0
#word_to_id["<START>"] = 1
#word_to_id["<UNK>"] = 2

#id_to_word = {value:key for key,value in word_to_id.items()}
#print(' '.join(id_to_word[id] for id in train_x[0] ))

In [4]:
#word_to_id = keras.datasets.imdb.get_word_index()

In [5]:
#test = [id_to_word[ix] for ix in x_train[0]]

In [6]:
#sen = 'These films were great.'


#def text_cleaning(sen):
#    tokenizer = RegexpTokenizer(r'\w+')
#    tok = tokenizer.tokenize(sen)
#    
#    stop_words = set(stopwords.words('english'))
#    wo_stop_words = list(filter(lambda ts: ts.lower() not in stop_words, tok))

#    #lemm = WordNetLemmatizer()
#    #stem = [lemm.lemmatize(w) for w in wo_stop_words]

    
#    return wo_stop_words
#text_cleaning(' '.join(test))

In [7]:
#tokenizer = RegexpTokenizer(r'\w+')
#tok = tokenizer.tokenize(' '.join(test))

#lemm = WordNetLemmatizer()
#stem = [lemm.lemmatize(t) for t in tok]

In [8]:
#tok

In [9]:
#one-hot encoding of sentences
tokenizer = Tokenizer(num_words=1000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

In [13]:
num_classes = 2
y_train2 = to_categorical(y_train, num_classes)
y_test2 = to_categorical(y_test, num_classes)

In [15]:
# Neural Network model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(x_train, y_train2, epochs=2, batch_size=100, verbose=1, validation_split=0.2)

# Evaluate the model
score = model.evaluate(x_train, y_train2)
print("\n Training Accuracy:", score[1])
score = model.evaluate(x_test, y_test2)
print("\n Testing Accuracy:", score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               128128    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 258       
Total params: 128,386
Trainable params: 128,386
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 2s 95us/step - loss: 0.4341 - acc: 0.7943 - val_loss: 0.3337 - val_acc: 0.8584
Epoch 2/2
25000/25000 [==============================] - 2s 66us/step
('\n Training Accuracy:', 0.8864)
25000/25000 [==============================] - 2s 81us/step
('\n Testing Accuracy:', 0.85952)


In [24]:
# define model
model = Sequential()
e = Embedding(x_train.shape[1], 100, input_length=x_train.shape[1])
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(x_train, y_train, epochs=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 1000, 100)         100000    
_________________________________________________________________
flatten_7 (Flatten)          (None, 100000)            0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 100001    
Total params: 200,001
Trainable params: 200,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
25000/25000 [==============================] - 17s 696us/step - loss: 0.3799 - acc: 0.8314
Epoch 2/50
25000/25000 [==============================] - 16s 625us/step - loss: 0.3275 - acc: 0.8621
Epoch 3/50
25000/25000 [==============================] - 15s 607us/step - loss: 0.3204 - acc: 0.8644
Epoch 4/50
25000/25000 [==============================] - 15s 602us/step - loss: 0.3173 -

NameError: name 'padded_docs' is not defined

In [26]:
# evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 85.740000


In [99]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = y_train#array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = x_train#t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = x_train#pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]
Loaded 400000 word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
None
Accuracy: 100.000000


In [103]:
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(x_train, y_train, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(x_train, y_train, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Loaded 400000 word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
None


ValueError: Error when checking input: expected embedding_2_input to have shape (4,) but got array with shape (1,)

In [111]:
x_train.shape

(25000, 1000)